In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/couplet-test-data/out.txt
/kaggle/input/couplet-test-data/in.txt


In [1]:
import torch
import torch.nn as nn
import math
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import OrderedDict
from torch.utils.data import Dataset
from tqdm import tqdm

# 位置编码
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size, dropout, maxlen=5000):
        super().__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros(maxlen, emb_size)
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        # 添加批次维度
        pos_embedding = pos_embedding.unsqueeze(-2)
        self.dropout = nn.Dropout(dropout)
        # positional encoding注册为不需要作为模型参数的缓冲中
        self.register_buffer('pos_embedding', pos_embedding)
    def forward(self, token_embedding):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# 将输入索引张量转换为token embedding张量
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, emb_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)
    
class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers, num_decoder_layers, emb_size, nhead, src_vocab_size, tgt_vocab_size, 
                 dim_feedforward=512, dropout=0.1):
        super().__init__()
        self.transformer = nn.Transformer(d_model=emb_size,
                                          nhead=nhead,
                                          num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers,
                                          dim_feedforward=dim_feedforward,
                                          dropout=dropout,
                                          batch_first=True)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_token_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_token_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout)
    def forward(self, src, tgt, src_mask, tgt_mask, src_key_padding_mask, tgt_key_padding_mask, memory_key_padding_mask):
        src_emb = self.positional_encoding(self.src_token_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_token_emb(tgt))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, 
                                src_key_padding_mask, tgt_key_padding_mask, memory_key_padding_mask)
        return self.generator(outs)
    def encode(self, src, src_mask):
        return self.transformer.encoder(self.positional_encoding(self.src_token_emb(src)), src_mask)
    def decode(self, tgt, memory, tgt_mask):
        return self.transformer.decoder(self.positional_encoding(self.tgt_token_emb(tgt)), memory, tgt_mask)

# 自定义数据集
class Seq2SeqDataset(Dataset):
    def __init__(self, encode_datas, decode_datas):
        super().__init__()
        self.encode_datas = encode_datas
        self.decode_datas = decode_datas
        self.encode_vocab = self.build_vocab(encode_datas, fill_mask = ['PAD', 'EOS', 'UNK'])
        self.decode_vocab = self.build_vocab(decode_datas, fill_mask = ['PAD', 'BOS', 'EOS', 'UNK'])
    def __getitem__(self, index):
        enc = list(self.encode_datas[index]) + ['EOS']
        dec = ['BOS'] + list(self.decode_datas[index]) + ['EOS']
        e = [self.encode_vocab.get(tk, self.encode_vocab['UNK']) for tk in enc]
        d = [self.decode_vocab.get(tk, self.decode_vocab['UNK']) for tk in dec]
        return e,d
    def __len__(self):
        return len(self.encode_datas)
    # 构建词汇表
    def build_vocab(self, datas, fill_mask):
        vocab = OrderedDict({msk: idx for idx, msk in enumerate(fill_mask)})
        for item in datas:
            for token in list(item):
                vocab[token] = vocab.get(token, len(vocab))
        return vocab

def read_couplet(path):
    datas = []
    with open(path, encoding='utf-8') as f:
        lines = f.readlines()
        for l in lines:
            datas.append(l.strip().split())
    return datas

def build_dataloader(dataset, batch_size, shuffle = False):
    def collate_batch(batch):
        encode_list, decode_list = [], []
        for encode, decode in batch:
            encode_list.append(torch.tensor(encode, dtype=torch.int64))
            decode_list.append(torch.tensor(decode, dtype=torch.int64))
        encode_list = pad_sequence(encode_list, batch_first=True, padding_value=0)
        decode_list = pad_sequence(decode_list, batch_first=True, padding_value=0)
        return encode_list, decode_list
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_batch)
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz))) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask
def create_mask(src, tgt):
    src_seq_len = src.shape[1]
    tgt_seq_len = tgt.shape[1]
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len)).type(torch.bool)
    src_padding_mask = src == 0
    tgt_padding_mask = tgt == 0
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask
# 训练模型
def train_epoch(epoch, train_dl, model, loss_fn, optimizer):
    model.train()
    losses = 0
    train_bar = tqdm(train_dl)
    for src, tgt in train_bar:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)
        tgt_input = tgt[:, :-1]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = src_mask.to(DEVICE), tgt_mask.to(DEVICE), src_padding_mask.to(DEVICE), tgt_padding_mask.to(DEVICE)
        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)
        optimizer.zero_grad()
        tgt_output = tgt[:, 1:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_output.reshape(-1))
        loss.backward()
        optimizer.step()
        losses += loss.item()
        train_bar.set_description(f"Epoch {epoch + 1}, Loss: {loss.item()}")
    avg_loss = losses/len(train_dl)
    print(f'epoch={epoch + 1}, avg_loss={avg_loss}')
    return losses/len(train_dl)

In [2]:
encode_datas = read_couplet('/kaggle/input/couplet-test-data/in.txt')
decode_datas = read_couplet('/kaggle/input/couplet-test-data/out.txt')
dataset = Seq2SeqDataset(encode_datas, decode_datas)
# 初始化超参数
LR = 0.0001
SRC_VOCAB_SIZE = len(dataset.encode_vocab)
TGT_VOCAB_SIZE = len(dataset.decode_vocab)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 2048
BATCH_SIZE = 64
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
NUM_EPOCHS = 60
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 模型初始化
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, 
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
transformer = transformer.to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(transformer.parameters(), lr=LR)
# 训练模型
train_dl = build_dataloader(dataset, BATCH_SIZE)
for epoch in range(NUM_EPOCHS):
    train_epoch(epoch, train_dl, transformer, loss_fn, optimizer)
torch.save(transformer.state_dict(), 'model_new.pth')

  0%|          | 0/63 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5849: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 1, Loss: 2.8482983112335205: 100%|██████████| 63/63 [00:06<00:00, 10.09it/s]


epoch=1, avg_loss=3.1825558533744207


Epoch 2, Loss: 2.74883770942688: 100%|██████████| 63/63 [00:05<00:00, 11.34it/s]  


epoch=2, avg_loss=2.6722925466204446


Epoch 3, Loss: 2.684967041015625: 100%|██████████| 63/63 [00:05<00:00, 11.17it/s] 


...

epoch=58, avg_loss=0.20570194650264012


Epoch 59, Loss: 0.06385817378759384: 100%|██████████| 63/63 [00:06<00:00,  9.40it/s]


epoch=59, avg_loss=0.19258426780265475


Epoch 60, Loss: 0.05539719760417938: 100%|██████████| 63/63 [00:06<00:00,  9.39it/s]


epoch=60, avg_loss=0.18152227997779846


In [17]:
# 推理
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src, src_mask = src.to(DEVICE), src_mask.to(DEVICE)
    memory = model.encode(src, src_mask)
    # decode的第一个token：BOS
    ys = torch.ones(1,1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.shape[1])).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()
        ys = torch.cat([ys, torch.ones(1,1).fill_(next_word).to(DEVICE)], dim=1)
        if next_word == dataset.decode_vocab['EOS']:
            break
    return ys
def translate(model, src_sentence):
    model.eval()
    src = torch.tensor([dataset.encode_vocab[tk] for tk in list(src_sentence) + ['EOS']]).reshape(1, -1)
    num_tokens = src.shape[1]
    src_mask = torch.zeros(num_tokens, num_tokens).type(torch.bool)
    tgt_tokens = greedy_decode(model, src, src_mask, num_tokens, dataset.decode_vocab['BOS']).reshape(-1)
    return ''.join([decode_vocab_rev[tk] for tk in list(tgt_tokens.cpu().numpy())][1:-1])

In [18]:
decode_vocab_rev = {v:k for k, v in dataset.decode_vocab.items()}
# transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, 
#                                      NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
# transformer = transformer.to(DEVICE)
# transformer.load_state_dict(torch.load("kaggle_output/model_new.pth"))
sen_enc = '陆才吟岁，心定方知时日快'
sen_dec = translate(transformer, sen_enc)
print(sen_enc)
print(sen_dec)

陆才吟岁，心定方知时日快
果真情人常是，应当达民族
